# 🌲 Chapter 5 练习 2: 因果森林 - 专为因果推断设计的「森林」

## 从随机森林到因果森林

上一节我们用 T-Learner 估计 CATE，但它有一个问题：**它不是为因果推断设计的**。

**因果森林 (Causal Forest)** 是 Athey & Imbens (2018) 提出的方法，专门为估计**异质性处理效应**设计，有更好的统计性质！

---

## 🎯 学习目标

1. 理解因果森林的核心原理
2. 理解「诚实分裂」的概念
3. 使用 econml 实现因果森林
4. 分析特征重要性

## 🌟 因果森林的直觉

### 随机森林 vs 因果森林

| 随机森林 | 因果森林 |
|----------|----------|
| 目标: 预测 Y | 目标: 估计 CATE |
| 最小化预测误差 | 最大化效应异质性 |
| 分裂准则: 减少方差 | 分裂准则: 最大化子节点间的 CATE 差异 |
| 同一数据用于分裂和预测 | **诚实分裂**: 不同数据用于分裂和估计 |

### 诚实分裂 (Honest Splitting)

这是因果森林的**核心创新**！

普通决策树：
```
同一批数据 -> 用于构建树结构
           -> 也用于叶节点预测
```

因果森林：
```
数据分成两半:
  - 分裂样本 (50%): 用于构建树结构（决定如何分裂）
  - 估计样本 (50%): 用于叶节点的 CATE 估计
```

### 为什么需要诚实分裂？

想象你在做投资回测：

- **不诚实**: 用过去数据找规律，又用同样的数据验证 → 过拟合！
- **诚实**: 用一半数据找规律，用另一半验证 → 更可靠！

因果森林的诚实分裂确保了**渐近正态性**和**有效的置信区间**！

## 📐 核心公式

### 因果森林的 CATE 估计

对于一个新样本 $x$，因果森林的预测是：

$$\hat{\tau}(x) = \frac{1}{B} \sum_{b=1}^{B} \hat{\tau}_b(x)$$

其中 $B$ 是树的数量，$\hat{\tau}_b(x)$ 是第 $b$ 棵树的预测。

### 单棵因果树的叶节点估计

在叶节点 $L$ 中：

$$\hat{\tau}_L = \frac{\sum_{i \in L, T_i=1} Y_i}{|\{i \in L: T_i=1\}|} - \frac{\sum_{i \in L, T_i=0} Y_i}{|\{i \in L: T_i=0\}|}$$

简单说：**叶节点内的处理组平均 Y 减去控制组平均 Y**

### 分裂准则

因果森林使用的分裂准则最大化子节点之间的 CATE 异质性：

$$\text{Criterion} = (\hat{\tau}_{\text{left}} - \hat{\tau}_{\text{right}})^2 \cdot n_{\text{left}} \cdot n_{\text{right}}$$

## 📐 Causal Forest 核心数学推导

### Honest Splitting 原理

**传统决策树问题**:
- 用同一批数据 $S$ 构建树结构
- 又用 $S$ 估计叶节点值
- 结果: 过拟合，置信区间失效

**Honest Splitting 解决方案**:

将样本 $S$ 随机分为两个互斥子集:
- $S_{split}$: 分裂样本 (构建树结构)
- $S_{est}$: 估计样本 (叶节点估计)

$$|S_{split}| = |S_{est}| = \frac{n}{2}$$

### 分裂准则 (最大化异质性)

**目标**: 找到分裂点 $(j, s)$ 使得左右子节点的 CATE 差异最大。

**分裂增益**:
$$\Delta(\tau) = \frac{n_L n_R}{(n_L + n_R)^2} \times (\hat{\tau}_L - \hat{\tau}_R)^2$$

其中:
- $n_L, n_R$: 左右节点样本数
- $\hat{\tau}_L, \hat{\tau}_R$: 左右节点的 CATE 估计

**归一化因子** $\frac{n_L n_R}{(n_L + n_R)^2}$:
- 平衡左右节点大小
- 当 $n_L = n_R = \frac{n}{2}$ 时最大
- 避免极端不平衡分裂

### 叶节点 CATE 估计

对于叶节点 $L$ (只使用 $S_{est}$ 中落入该叶的样本):

$$\hat{\tau}_L = \bar{Y}_{1,L} - \bar{Y}_{0,L}$$

其中:
$$\bar{Y}_{1,L} = \frac{1}{n_{1,L}} \sum_{i \in L, T_i=1} Y_i$$
$$\bar{Y}_{0,L} = \frac{1}{n_{0,L}} \sum_{i \in L, T_i=0} Y_i$$

### 置信区间

因为使用了 Honest Splitting，$\hat{\tau}_L$ 是渐近正态的:

$$\hat{\tau}_L \sim N\left(\tau_L, \frac{\sigma^2_{1,L}}{n_{1,L}} + \frac{\sigma^2_{0,L}}{n_{0,L}}\right)$$

**95% 置信区间**:
$$\text{CI}_{95\%} = \hat{\tau}_L \pm 1.96 \sqrt{\frac{\hat{\sigma}^2_{1,L}}{n_{1,L}} + \frac{\hat{\sigma}^2_{0,L}}{n_{0,L}}}$$

其中 $\hat{\sigma}^2_{t,L}$ 是叶节点内的样本方差估计。

### Causal Forest 的集成

单棵树方差大，所以集成 $B$ 棵树:

$$\hat{\tau}_{CF}(x) = \frac{1}{B} \sum_{b=1}^{B} \hat{\tau}_b(x)$$

每棵树:
1. 随机抽样 bootstrap 样本
2. 随机选择特征子集 (MTRY)
3. Honest Splitting 构建树
4. 估计 CATE

**方差减少**:
$$Var[\hat{\tau}_{CF}] \approx \frac{Var[\hat{\tau}_{tree}]}{B}$$

### 与 Random Forest 的区别

| 特性 | Random Forest | Causal Forest |
|------|---------------|---------------|
| 目标 | 预测 $Y$ | 估计 $\tau(x)$ |
| 分裂准则 | MSE 减少 | CATE 异质性最大化 |
| Honest Splitting | 否 | 是 |
| 叶节点值 | $\bar{Y}$ | $\bar{Y}_1 - \bar{Y}_0$ |
| 置信区间 | 不可靠 | 理论保证 |

In [ ]:
# 环境设置
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Tuple, Optional
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# 设置随机种子
np.random.seed(42)

# 尝试导入 econml
try:
    from econml.grf import CausalForest
    ECONML_AVAILABLE = True
    print("econml 导入成功! ✅")
except ImportError:
    ECONML_AVAILABLE = False
    print("警告: econml 未安装 ⚠️")
    print("安装命令: pip install econml")

# 绘图设置
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['font.size'] = 12
plt.rcParams['axes.unicode_minus'] = False
plt.style.use('seaborn-v0_8-whitegrid')

print("\n环境配置完成! 🌲")

def honest_split_data(
    X: np.ndarray,
    T: np.ndarray,
    Y: np.ndarray,
    split_ratio: float = 0.5,
    seed: int = 42
) -> Tuple[Tuple, Tuple]:
    """
    诚实分裂: 将数据分为两部分
    
    - 分裂样本: 用于构建树结构（决定在哪里分裂）
    - 估计样本: 用于叶节点的 CATE 估计（计算实际效应）
    
    这是因果森林的核心创新，防止过拟合！
    
    Args:
        X, T, Y: 特征、处理、结果
        split_ratio: 分裂样本的比例
        seed: 随机种子
    
    Returns:
        ((X_split, T_split, Y_split), (X_est, T_est, Y_est))
    """
    np.random.seed(seed)
    n = len(X)
    
    # 随机打乱索引
    indices = np.arange(n)
    np.random.shuffle(indices)
    
    # 计算分裂点
    split_point = int(n * split_ratio)
    
    # 划分索引
    split_idx = indices[:split_point]
    est_idx = indices[split_point:]
    
    # 根据索引划分数据
    # 分裂样本
    X_split = X[split_idx]
    T_split = T[split_idx]
    Y_split = Y[split_idx]
    
    # 估计样本
    X_est = X[est_idx]
    T_est = T[est_idx]
    Y_est = Y[est_idx]
    
    return ((X_split, T_split, Y_split), (X_est, T_est, Y_est))

In [ ]:
def honest_split_data(
    X: np.ndarray,
    T: np.ndarray,
    Y: np.ndarray,
    split_ratio: float = 0.5,
    seed: int = 42
) -> Tuple[Tuple, Tuple]:
    """
    诚实分裂: 将数据分为两部分

    - 分裂样本: 用于构建树结构（决定在哪里分裂）
    - 估计样本: 用于叶节点的 CATE 估计（计算实际效应）

    这是因果森林的核心创新，防止过拟合！

    Args:
        X, T, Y: 特征、处理、结果
        split_ratio: 分裂样本的比例
        seed: 随机种子

    Returns:
        ((X_split, T_split, Y_split), (X_est, T_est, Y_est))
    """
    np.random.seed(seed)
    n = len(X)

    # 随机打乱索引
    indices = np.arange(n)
    np.random.shuffle(indices)

    # 计算分裂点
    split_point = int(n * split_ratio)

    # 划分索引
    split_idx = indices[:split_point]
    est_idx = indices[split_point:]

    # 根据索引划分数据
    # 分裂样本
    X_split = X[split_idx]
    T_split = T[split_idx]
    Y_split = Y[split_idx]

    # 估计样本
    X_est = X[est_idx]
    T_est = T[est_idx]
    Y_est = Y[est_idx]

    return ((X_split, T_split, Y_split), (X_est, T_est, Y_est))

In [ ]:
# 生成测试数据
def generate_test_data(n: int = 2000, seed: int = 42):
    """
    生成测试数据
    
    异质性效应: tau = 3 + 2*X1 - 1.5*X2
    """
    np.random.seed(seed)
    
    # 特征
    X1 = np.random.randn(n)
    X2 = np.random.randn(n)
    X3 = np.random.randn(n)  # 噪声特征，不影响 CATE
    
    X = np.column_stack([X1, X2, X3])
    
    # 随机处理
    T = np.random.binomial(1, 0.5, n)
    
    # 异质性效应
    tau = 3.0 + 2.0 * X1 - 1.5 * X2
    
    # 结果
    Y0 = 10 + 1.5 * X1 + 1.0 * X2 + 0.5 * X3 + np.random.randn(n)
    Y1 = Y0 + tau
    Y = np.where(T == 1, Y1, Y0)
    
    return X, T, Y, tau


# 生成数据
X, T, Y, tau_true = generate_test_data(n=2000)

print("数据生成成功! 🎉")
print(f"数据形状: X={X.shape}, T={T.shape}, Y={Y.shape}")
print(f"处理比例: {T.mean():.2%}")
print(f"真实 ATE: {tau_true.mean():.4f}")
print(f"真实 CATE 范围: [{tau_true.min():.2f}, {tau_true.max():.2f}]")

In [ ]:
# 测试诚实分裂
try:
    split_data = honest_split_data(X, T, Y, split_ratio=0.5)
    
    if split_data is not None:
        (X_split, T_split, Y_split), (X_est, T_est, Y_est) = split_data
        
        if X_split is not None:
            print("诚实分裂测试通过! 🎉")
            print(f"\n分裂样本: {len(X_split)} 个")
            print(f"  - 处理组: {T_split.sum()}, 控制组: {(1-T_split).sum()}")
            print(f"\n估计样本: {len(X_est)} 个")
            print(f"  - 处理组: {T_est.sum()}, 控制组: {(1-T_est).sum()}")
            
            print(f"\n诚实分裂的意义:")
            print(f"  - 分裂样本用于决定树的结构（哪些特征重要）")
            print(f"  - 估计样本用于计算叶节点的 CATE（防止过拟合）")
        else:
            print("[TODO] 请完成数据划分部分")
    else:
        print("[TODO] 请完成 honest_split_data 函数")
except Exception as e:
    print(f"[错误] {e}")

def train_causal_forest(
    X: np.ndarray,
    T: np.ndarray,
    Y: np.ndarray,
    n_estimators: int = 100,
    min_samples_leaf: int = 5,
    max_depth: int = None
) -> Optional[object]:
    """
    使用 econml 训练因果森林
    
    Args:
        X: 特征矩阵
        T: 处理变量 (0/1)
        Y: 结果变量
        n_estimators: 树的数量
        min_samples_leaf: 叶节点最小样本数
        max_depth: 最大深度
    
    Returns:
        训练好的因果森林模型
    """
    if not ECONML_AVAILABLE:
        print("econml 不可用")
        return None
    
    # 创建 CausalForest 实例
    cf = CausalForest(
        n_estimators=n_estimators,
        min_samples_leaf=min_samples_leaf,
        max_depth=max_depth,
        honest=True,  # 使用诚实分裂
        random_state=42
    )
    
    # 训练模型
    # 注意: econml 的 fit 方法是 fit(Y, T, X=X)
    cf.fit(Y, T, X=X)
    
    return cf

In [ ]:
def train_causal_forest(
    X: np.ndarray,
    T: np.ndarray,
    Y: np.ndarray,
    n_estimators: int = 100,
    min_samples_leaf: int = 5,
    max_depth: int = None
) -> Optional[object]:
    """
    使用 econml 训练因果森林

    Args:
        X: 特征矩阵
        T: 处理变量 (0/1)
        Y: 结果变量
        n_estimators: 树的数量
        min_samples_leaf: 叶节点最小样本数
        max_depth: 最大深度

    Returns:
        训练好的因果森林模型
    """
    if not ECONML_AVAILABLE:
        print("econml 不可用")
        return None

    # 创建 CausalForest 实例
    cf = CausalForest(
        n_estimators=n_estimators,
        min_samples_leaf=min_samples_leaf,
        max_depth=max_depth,
        honest=True,  # 使用诚实分裂
        random_state=42
    )

    # 训练模型
    # 注意: econml 的 fit 方法是 fit(Y, T, X=X)
    cf.fit(Y, T, X=X)

    return cf

In [ ]:
# 分割训练/测试集
train_idx = np.random.choice(len(X), int(0.7 * len(X)), replace=False)
test_idx = np.array([i for i in range(len(X)) if i not in train_idx])

X_train, T_train, Y_train = X[train_idx], T[train_idx], Y[train_idx]
X_test, T_test, Y_test = X[test_idx], T[test_idx], Y[test_idx]
tau_test = tau_true[test_idx]

print(f"训练集: {len(X_train)} 样本")
print(f"测试集: {len(X_test)} 样本")

In [ ]:
# 训练因果森林
if ECONML_AVAILABLE:
    print("训练因果森林...")
    
    try:
        cf = train_causal_forest(
            X_train, T_train, Y_train,
            n_estimators=100,
            min_samples_leaf=5
        )
        
        if cf is not None:
            # 预测 CATE
            tau_pred_cf = cf.predict(X_test).flatten()
            
            # 计算 PEHE
            pehe_cf = np.sqrt(np.mean((tau_test - tau_pred_cf) ** 2))
            
            # 计算相关性
            corr_cf = np.corrcoef(tau_test, tau_pred_cf)[0, 1]
            
            print("\n因果森林训练完成! 🌲")
            print(f"\n评估结果:")
            print(f"  PEHE: {pehe_cf:.4f}")
            print(f"  相关性: {corr_cf:.4f}")
            print(f"  ATE 预测: {tau_pred_cf.mean():.4f} (真实: {tau_test.mean():.4f})")
        else:
            print("[TODO] 请完成 train_causal_forest 函数")
    except Exception as e:
        print(f"[错误] {e}")
        import traceback
        traceback.print_exc()
else:
    print("请安装 econml: pip install econml")

def compare_models(
    X_train: np.ndarray,
    T_train: np.ndarray,
    Y_train: np.ndarray,
    X_test: np.ndarray,
    tau_test: np.ndarray
) -> pd.DataFrame:
    """
    对比不同模型的 CATE 估计性能
    
    模型:
    1. T-Learner (Random Forest)
    2. Causal Forest (如果可用)
    """
    results = []
    predictions = {}
    
    # ===== 模型 1: T-Learner =====
    print("训练 T-Learner...")
    
    # 训练 T-Learner
    model_0 = RandomForestRegressor(n_estimators=100, random_state=42)
    model_1 = RandomForestRegressor(n_estimators=100, random_state=43)
    
    # 分别训练处理组和对照组模型
    mask_0 = (T_train == 0)
    mask_1 = (T_train == 1)
    
    model_0.fit(X_train[mask_0], Y_train[mask_0])
    model_1.fit(X_train[mask_1], Y_train[mask_1])
    
    # 预测 CATE
    tau_pred_tlearner = model_1.predict(X_test) - model_0.predict(X_test)
    
    if tau_pred_tlearner is not None:
        pehe_tlearner = np.sqrt(np.mean((tau_test - tau_pred_tlearner)**2))
        corr_tlearner = np.corrcoef(tau_test, tau_pred_tlearner)[0, 1]
        
        results.append({
            'Model': 'T-Learner',
            'PEHE': pehe_tlearner,
            'Correlation': corr_tlearner,
            'ATE Error': abs(tau_test.mean() - tau_pred_tlearner.mean())
        })
        predictions['T-Learner'] = tau_pred_tlearner
    
    # ===== 模型 2: Causal Forest =====
    if ECONML_AVAILABLE:
        print("训练 Causal Forest...")
        
        try:
            cf = train_causal_forest(X_train, T_train, Y_train)
            
            if cf is not None:
                tau_pred_cf = cf.predict(X_test).flatten()
                pehe_cf = np.sqrt(np.mean((tau_test - tau_pred_cf)**2))
                corr_cf = np.corrcoef(tau_test, tau_pred_cf)[0, 1]
                
                results.append({
                    'Model': 'Causal Forest',
                    'PEHE': pehe_cf,
                    'Correlation': corr_cf,
                    'ATE Error': abs(tau_test.mean() - tau_pred_cf.mean())
                })
                predictions['Causal Forest'] = tau_pred_cf
        except Exception as e:
            print(f"Causal Forest 训练失败: {e}")
    
    return pd.DataFrame(results), predictions

In [ ]:
def compare_models(
    X_train: np.ndarray,
    T_train: np.ndarray,
    Y_train: np.ndarray,
    X_test: np.ndarray,
    tau_test: np.ndarray
) -> pd.DataFrame:
    """
    对比不同模型的 CATE 估计性能

    模型:
    1. T-Learner (Random Forest)
    2. Causal Forest (如果可用)
    """
    results = []
    predictions = {}

    # ===== 模型 1: T-Learner =====
    print("训练 T-Learner...")

    # 训练 T-Learner
    model_0 = RandomForestRegressor(n_estimators=100, random_state=42)
    model_1 = RandomForestRegressor(n_estimators=100, random_state=43)

    # 分别训练处理组和对照组模型
    mask_0 = (T_train == 0)
    mask_1 = (T_train == 1)

    model_0.fit(X_train[mask_0], Y_train[mask_0])
    model_1.fit(X_train[mask_1], Y_train[mask_1])

    # 预测 CATE
    tau_pred_tlearner = model_1.predict(X_test) - model_0.predict(X_test)

    if tau_pred_tlearner is not None:
        pehe_tlearner = np.sqrt(np.mean((tau_test - tau_pred_tlearner)**2))
        corr_tlearner = np.corrcoef(tau_test, tau_pred_tlearner)[0, 1]

        results.append({
            'Model': 'T-Learner',
            'PEHE': pehe_tlearner,
            'Correlation': corr_tlearner,
            'ATE Error': abs(tau_test.mean() - tau_pred_tlearner.mean())
        })
        predictions['T-Learner'] = tau_pred_tlearner

    # ===== 模型 2: Causal Forest =====
    if ECONML_AVAILABLE:
        print("训练 Causal Forest...")

        try:
            cf = train_causal_forest(X_train, T_train, Y_train)

            if cf is not None:
                tau_pred_cf = cf.predict(X_test).flatten()
                pehe_cf = np.sqrt(np.mean((tau_test - tau_pred_cf)**2))
                corr_cf = np.corrcoef(tau_test, tau_pred_cf)[0, 1]

                results.append({
                    'Model': 'Causal Forest',
                    'PEHE': pehe_cf,
                    'Correlation': corr_cf,
                    'ATE Error': abs(tau_test.mean() - tau_pred_cf.mean())
                })
                predictions['Causal Forest'] = tau_pred_cf
        except Exception as e:
            print(f"Causal Forest 训练失败: {e}")

    return pd.DataFrame(results), predictions

In [ ]:
# 模型对比
print("=" * 60)
print("模型对比实验")
print("=" * 60)

try:
    comparison_df, predictions = compare_models(
        X_train, T_train, Y_train,
        X_test, tau_test
    )
    
    if comparison_df is not None and len(comparison_df) > 0:
        print("\n" + "=" * 60)
        print("对比结果:")
        print("=" * 60)
        display(comparison_df.round(4))
        
        print("\n说明:")
        print("- PEHE: 越小越好 (Precision in Estimation of HTE)")
        print("- Correlation: 越接近 1 越好 (CATE 排序能力)")
        print("- ATE Error: 越小越好 (平均效应估计误差)")
    else:
        print("[TODO] 请完成 compare_models 函数")
except Exception as e:
    print(f"[错误] {e}")

In [ ]:
# 可视化对比
if 'predictions' in dir() and len(predictions) > 0:
    n_models = len(predictions)
    fig, axes = plt.subplots(1, n_models, figsize=(7*n_models, 6))
    
    if n_models == 1:
        axes = [axes]
    
    for ax, (name, tau_pred) in zip(axes, predictions.items()):
        ax.scatter(tau_test, tau_pred, alpha=0.3)
        
        # 完美预测线
        min_val = min(tau_test.min(), tau_pred.min())
        max_val = max(tau_test.max(), tau_pred.max())
        ax.plot([min_val, max_val], [min_val, max_val], 'r--', label='完美预测')
        
        # 计算 PEHE
        pehe = np.sqrt(np.mean((tau_test - tau_pred)**2))
        corr = np.corrcoef(tau_test, tau_pred)[0, 1]
        
        ax.set_xlabel('真实 CATE')
        ax.set_ylabel('预测 CATE')
        ax.set_title(f'{name}\nPEHE={pehe:.4f}, Corr={corr:.4f}')
        ax.legend()
    
    plt.tight_layout()
    plt.show()

## 练习 2.4: 特征重要性分析

因果森林可以告诉我们：**哪些特征对 CATE 的异质性贡献最大？**

这与预测模型的特征重要性不同：
- 预测模型: 哪些特征对预测 Y 重要？
- 因果森林: 哪些特征导致处理效应不同？

In [ ]:
def get_feature_importances(
    model,
    feature_names: list
) -> pd.DataFrame:
    """
    获取因果森林的特征重要性

    Returns:
        DataFrame with columns: feature, importance
    """
    # 获取特征重要性
    try:
        importances = model.feature_importances()
    except:
        importances = getattr(model, 'feature_importances_', None)

    if importances is None:
        return None

    # 创建 DataFrame 并排序
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importances.flatten()
    })

    # 按重要性排序
    importance_df = importance_df.sort_values('Importance', ascending=False)

    return importance_df

In [ ]:
# 特征重要性分析
if ECONML_AVAILABLE and 'cf' in dir() and cf is not None:
    feature_names = ['X1', 'X2', 'X3']
    
    try:
        importance_df = get_feature_importances(cf, feature_names)
        
        if importance_df is not None:
            print("特征重要性分析:")
            print("="*40)
            display(importance_df.round(4))
            
            print("\n回顾数据生成过程:")
            print("  tau = 3.0 + 2.0*X1 - 1.5*X2")
            print("  X3 是噪声特征，不影响 CATE")
            print("\n预期: X1 和 X2 重要性高，X3 重要性低")
            
            # 可视化
            fig, ax = plt.subplots(figsize=(10, 5))
            
            colors = ['#3498db', '#e74c3c', '#95a5a6']  # X3 用灰色表示噪声
            bars = ax.barh(
                importance_df['Feature'], 
                importance_df['Importance'],
                color=[colors[feature_names.index(f)] for f in importance_df['Feature']]
            )
            
            ax.set_xlabel('重要性')
            ax.set_title('因果森林特征重要性\n(对 CATE 异质性的贡献)')
            
            # 添加数值标签
            for bar, val in zip(bars, importance_df['Importance']):
                ax.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2,
                       f'{val:.3f}', va='center')
            
            plt.tight_layout()
            plt.show()
        else:
            print("无法获取特征重要性")
    except Exception as e:
        print(f"特征重要性分析失败: {e}")
else:
    print("需要先训练因果森林模型")

## 练习 2.5: 置信区间

因果森林的另一个优点：可以提供**有效的置信区间**！

这是诚实分裂带来的统计性质，让我们可以进行推断。

In [ ]:
# 获取置信区间
if ECONML_AVAILABLE and 'cf' in dir() and cf is not None:
    try:
        # 获取预测和置信区间
        tau_pred = cf.predict(X_test).flatten()
        
        # 尝试获取置信区间
        try:
            tau_lower, tau_upper = cf.predict_interval(X_test, alpha=0.05)
            tau_lower = tau_lower.flatten()
            tau_upper = tau_upper.flatten()
            has_ci = True
        except:
            has_ci = False
            print("模型不支持置信区间")
        
        if has_ci:
            print("置信区间分析:")
            print("="*50)
            
            # 计算覆盖率
            coverage = np.mean((tau_test >= tau_lower) & (tau_test <= tau_upper))
            print(f"95% 置信区间覆盖率: {coverage:.2%}")
            print(f"(理论上应该接近 95%)")
            
            # 可视化
            fig, axes = plt.subplots(1, 2, figsize=(14, 5))
            
            # 选择一些样本可视化
            sample_idx = np.random.choice(len(tau_pred), 50, replace=False)
            sample_idx = np.sort(sample_idx)
            
            # 图1: 带置信区间的预测
            x_pos = range(len(sample_idx))
            axes[0].errorbar(
                x_pos, tau_pred[sample_idx],
                yerr=[tau_pred[sample_idx] - tau_lower[sample_idx],
                      tau_upper[sample_idx] - tau_pred[sample_idx]],
                fmt='o', capsize=3, label='预测 ± 95% CI'
            )
            axes[0].scatter(x_pos, tau_test[sample_idx], color='red', 
                           marker='x', s=50, label='真实值')
            axes[0].set_xlabel('样本索引')
            axes[0].set_ylabel('CATE')
            axes[0].set_title('CATE 预测与 95% 置信区间')
            axes[0].legend()
            
            # 图2: 置信区间宽度分布
            ci_width = tau_upper - tau_lower
            axes[1].hist(ci_width, bins=30, alpha=0.7, color='#2ecc71', edgecolor='black')
            axes[1].axvline(x=ci_width.mean(), color='red', linestyle='--',
                           label=f'平均宽度: {ci_width.mean():.3f}')
            axes[1].set_xlabel('置信区间宽度')
            axes[1].set_ylabel('频数')
            axes[1].set_title('置信区间宽度分布')
            axes[1].legend()
            
            plt.tight_layout()
            plt.show()
    except Exception as e:
        print(f"置信区间分析失败: {e}")
else:
    print("需要先训练因果森林模型")

## 🤔 思考题

## 🎯 Causal Forest 面试高频题

### Q1: 什么是 Honest Splitting？为什么重要？

**标准答案**:

**定义**: 将样本分为两个不重叠的子集:
- 分裂样本: 决定树的结构
- 估计样本: 估计叶节点的 CATE

**为什么重要**:
1. **无偏性**: 避免选择偏差 (selection bias)
2. **置信区间**: 保证渐近正态性，使得置信区间有效
3. **防止过拟合**: 构建和估计使用不同数据

**代价**: 需要两倍样本量才能达到相同精度

### Q2: Causal Forest 的分裂准则是什么？

**标准答案**:

**目标**: 最大化子节点间的 CATE 异质性

**分裂增益**:
$$\Delta = \frac{n_L \cdot n_R}{(n_L + n_R)^2} \times (\hat{\tau}_L - \hat{\tau}_R)^2$$

**与 Random Forest 的区别**:
- RF: 最小化 MSE
- CF: 最大化 CATE 差异

**直觉**: 我们希望一个叶节点里的人效应都相似，但不同叶节点的效应差异大。

### Q3: Causal Forest vs T-Learner 有什么区别？

**标准答案**:

| 特性 | T-Learner | Causal Forest |
|------|-----------|---------------|
| 设计目标 | 通用 | 专为 CATE 设计 |
| 分裂准则 | 预测 Y | 最大化 CATE 异质性 |
| Honest Splitting | 无 | 有 |
| 置信区间 | 不可靠 | 有理论保证 |
| 特征重要性 | 对 Y 的重要性 | 对 CATE 异质性的贡献 |

**何时用哪个**:
- T-Learner: 快速原型，数据少
- Causal Forest: 需要置信区间，数据多

### Q4: 特征重要性在 Causal Forest 中代表什么？

**标准答案**:

**Causal Forest 的特征重要性** 衡量的是:
- 该特征对 **CATE 异质性** 的贡献
- 不是对预测 Y 的贡献！

**计算方式**:
- 基于该特征分裂带来的 CATE 差异增益总和
- 类似 RF 的 Gini importance，但目标不同

**实际意义**:
- 重要性高 → 该特征是效应异质性的主要驱动因素
- 可以指导业务理解"为什么不同人效应不同"

### Q5: Causal Forest 的计算复杂度如何？什么时候不应该用它？

**标准答案**:

**计算复杂度**:
- 训练: $O(B \cdot n \log n \cdot p)$ 其中 B=树数量, n=样本数, p=特征数
- 预测: $O(B \cdot \log n)$

**内存需求**: 比 T-Learner 高 (需要存储所有树)

**不适合的场景**:
1. 样本量很小 (< 500) - Honest Splitting 需要更多数据
2. 效应是简单线性关系 - 线性模型就够了
3. 计算资源有限 - T-Learner 更快
4. 只需要 ATE - 不需要 Causal Forest 的精细建模

**适合的场景**:
1. 需要置信区间和统计推断
2. 效应异质性复杂且非线性
3. 需要特征重要性分析
4. 数据量充足 (> 2000)

In [ ]:
# 思考题 1: 什么是诚实分裂 (Honest Splitting)？它如何防止过拟合？

answer_1 = """
你的答案:



"""
print(answer_1)

In [ ]:
# 思考题 2: 因果森林与标准随机森林有何不同？为什么不能直接用随机森林估计 CATE？

answer_2 = """
你的答案:



"""
print(answer_2)

In [ ]:
# 思考题 3: 在什么情况下，因果森林比 T-Learner 表现更好？

answer_3 = """
你的答案:



"""
print(answer_3)

In [ ]:
# 思考题 4: 特征重要性在因果推断中如何解释？它与预测模型中的特征重要性有何不同？

answer_4 = """
你的答案:



"""
print(answer_4)

In [ ]:
# 思考题 5: 因果森林的计算复杂度如何？在什么情况下应该考虑使用更简单的方法？

answer_5 = """
你的答案:



"""
print(answer_5)

## 📊 总结

### 因果森林 vs T-Learner

| 方面 | T-Learner | 因果森林 |
|------|-----------|----------|
| 设计目的 | 通用预测方法的组合 | 专为 CATE 设计 |
| 分裂准则 | 预测 Y | 最大化 CATE 异质性 |
| 诚实分裂 | 无 | 有 |
| 置信区间 | 不直接支持 | 理论保证 |
| 特征重要性 | 对 Y 的重要性 | 对 CATE 异质性的重要性 |

### 因果森林核心概念

| 概念 | 说明 |
|------|------|
| 诚实分裂 | 用不同数据构建树和估计效应 |
| 分裂准则 | 最大化子节点 CATE 差异 |
| 叶节点估计 | 处理组均值 - 控制组均值 |
| 集成 | 多棵树的平均 |

### 什么时候用因果森林？

✅ **适合场景**:
- 需要可靠的置信区间
- 想了解哪些特征驱动效应异质性
- 数据量足够大（诚实分裂需要更多数据）

❌ **不适合场景**:
- 数据量很小
- 效应是简单的线性关系
- 计算资源有限

In [ ]:
print("🌲 恭喜完成因果森林练习!")
print("\n你已经学会了:")
print("  ✓ 诚实分裂的概念和实现")
print("  ✓ 使用 econml 训练因果森林")
print("  ✓ 因果森林 vs T-Learner 对比")
print("  ✓ 特征重要性分析")
print("  ✓ 置信区间估计")
print("\n下一步: 敏感性分析 - 检验因果结论的稳健性!")